In [1]:
# To default to float division and print function.
from __future__ import (division, print_function)

# Core python libraries
import csv
import re
import math

# External libraries.
import nltk
import numpy
import pandas
import scipy
import sklearn
# Sklearn is not eager in importing its dependancies. Have to explicity load them.
import sklearn.ensemble

# So you know when this code block finishes.
print ("Done")

Done


In [2]:
data = pandas.read_csv('data/train_medium.csv')

X_train_index, X_test_index, Y_train, y_test = sklearn.cross_validation.train_test_split(
    data.index, data['LogSalaryNormalized'], test_size=.33, random_state=42)

# Keep train and test as pandas dataframes.
X_train = data.iloc[X_train_index]
X_test = data.iloc[X_test_index]

data.head()


,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName,DescriptionLength,LogSalaryNormalized
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk,348,10.126631
1,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk,397,10.221941
2,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk,216,10.126631
3,13179816,Engineering Systems Analyst Water Industry,Engineering Systems Analyst Water Industry Loc...,"Dorking, Surrey, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20K to 30K,25000,cv-library.co.uk,1602,10.126631
4,14131336,Senior Subsea Pipeline Integrity Engineer,A globally renowned engineering and training c...,"Aberdeen, Borders",UK,NaN,permanent,Indigo 21 Ltd,Engineering Jobs,50000 - 100000/annum,75000,cv-library.co.uk,1364,11.225243


In [3]:
MIN_WORD_FREQUENCY = 10

count_vect = sklearn.feature_extraction.text.CountVectorizer(
  stop_words='english', min_df=MIN_WORD_FREQUENCY)

X_train_counts = count_vect.fit_transform(X_train['FullDescription'])
Y_train_counts =count_vect.transform(X_test['FullDescription'])

print(X_train_counts.shape)
print(Y_train_counts.shape)
#TODO (max): supress the VisibleDeprecationWarning here from numpy. It is being thrown by the package.

(670, 1100)
(330, 1100)


/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2645: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [4]:
def original_salary_scale(log_value):
    return math.exp(log_value)

# Guess the average. Create an empty vector of the desired shape.
average_guess = numpy.empty(y_test.shape)
average_guess.fill(numpy.mean(Y_train))

mean_guess = average_guess[0]
print('Guess value is %s on log scale or £%s' % (mean_guess, original_salary_scale(mean_guess)))
average_guess_mae = sklearn.metrics.mean_absolute_error(y_test, average_guess)
print('Guess the average Mean Absolute Error: {:10.4f}'.format(average_guess_mae))

# We want a stochastic gradient descent with l1 norm.
sgd = sklearn.linear_model.SGDRegressor(penalty='l1', n_iter=10)
sgd.fit(X_train_counts, Y_train)
sgd_predictions = sgd.predict(Y_train_counts)
sgd_mae = sklearn.metrics.mean_absolute_error(y_test, sgd_predictions)
print('SGDRegressor Mean Absolute Error: {:10.4f}'.format(sgd_mae))

# Random Forest.
rf = sklearn.ensemble.RandomForestRegressor(n_estimators=10)
rf.fit(X_train_counts, Y_train)
rf_predictions = rf.predict(Y_train_counts)

# Mean Absolute Error
rf_mae = sklearn.metrics.mean_absolute_error(y_test, rf_predictions)
print('Random Forest Regressor Mean Absolute Error: {:10.4f}'.format(rf_mae))


Guess value is 10.1635886033 on log scale or £25941.2229821
Guess the average Mean Absolute Error:     0.3341
SGDRegressor Mean Absolute Error:     1.9665
Random Forest Regressor Mean Absolute Error:     0.1613


In [5]:
import tensorflow



In [7]:
import tflearn

In [8]:
tflearn.datasets

<module 'tflearn.datasets' from '/usr/local/lib/python2.7/dist-packages/tflearn/datasets/__init__.pyc'>

In [27]:
#print (tensorflow.sparse_to_dense(X_train_counts[0], 670, X_train_counts[1]))



<tf.Tensor 'Const:0' shape=(670, 1100) dtype=int64>

In [90]:
# Please see http://tflearn.org/examples/
#dnn_model = tflearn.models.dnn.DNN([3,3,3,1], 0
X = X_train_counts.todense().astype(float)
m, n = numpy.shape(X)
Y = Y_train
Y = Y.reshape(m, 1).astype(float)
#print (Y.shape)


#input_layer = tflearn.input_data(shape=[None, n], name='input')
#linear = tflearn.fully_connected(input_layer, 16)

#dnn_regressor = tensorflow.contrib.learn.DNNRegressor(hidden_units=[128, 1])
#dnn_regressor.fit(feed_dict={X: X, Y: Y}, batch_size=100, steps=1000)

#regression = tflearn.regression(linear, optimizer='sgd', loss='mean_square',
#                                metric='R2', learning_rate=0.01)
#m = tflearn.DNN(regression)
#m.fit(X, Y, n_epoch=1000, show_metric=True, snapshot_epoch=False)


In [92]:

W = tensorflow.Variable(tensorflow.zeros([m,1]))
b = tensorflow.Variable(tensorflow.zeros([1]))

b = tensorflow.Print(b, [b], "Bias: ")
W = tensorflow.Print(W, [W], "Weights: ")

y = tensorflow.add(tensorflow.matmul(X, W), b)
y = tensorflow.Print(y, [y], "y: ")

loss = tensorflow.reduce_sum(tensorflow.square(y - y_data)) / (2 * m)
loss = tensorflow.Print(loss, [loss], "loss: ")
optimizer = tensorflow.train.GradientDescentOptimizer(.001)

train = optimizer.minimize(loss)

init = tensorflow.initialize_all_variables()

sess = tensorflow.Session()
sess.run(init)                                

for i in range(10):
  sess.run(train)
  #if i % 20 == 0:
        #print(sess.run(W), sess.run(b))  

sess.close()

TypeError: Input 'b' of 'MatMul' Op has type float32 that does not match type float64 of argument 'a'.